# Experiment 221 Adiabatenkoeffizient - Auswertung
##### Felix Fleischle - 8.11.2021

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

## 1. Bestimmung des Adiabatenkoeffizienten von Luft nach Clément und Desormes
Wir tragen zuerst unsere Messwerte für die Höhen in Arrays ein:

In [2]:
h_1 = np.array([0.08 , 0.079 , 0.121 , 0.119 , 0.085]) # meter
h_1_err = np.ones([5])*0.002
h_3 = np.array([0.012 , 0.012 , 0.056 , 0.026 , 0.038]) # meter
h_3_err = np.ones([5])*0.002

In [3]:
# Nun können wir den Adiabtenkoeffizienten bestimmen
kappa_cd = (h_1)/(h_1 - h_3)
kappa_cd_err = np.sqrt( ( (1 / (h_1 - h_3) + h_1 / (h_1 - h_3)**2) * h_1_err )**2 + (h_1 * h_3_err / (h_1 - h_3)**2)**2 )
print("Adiabatenkoeffizienten: ", kappa_cd)
print("Fehler Adiabatenkoeffizienten: ", kappa_cd_err)

# Wir bilden den Mittelwert
kappa_cd_mean = np.mean(kappa_cd)
kappa_cd_mean_std = np.std(kappa_cd)/np.sqrt(5)

print("Mittelwert Adiabatenkoeffizient: ", kappa_cd_mean)
print("Fehler Mittelwert: ", kappa_cd_mean_std)

# Wir schätzen den Fehler nach oben ab mit dem Mittelwert der systematischen Fehler der einzelnen Adiabatenkoeffizienten
kappa_cd_mean_sys = np.mean(kappa_cd_err)
kappa_cd_mean_err = np.sqrt(kappa_cd_mean_std**2 + kappa_cd_mean_sys**2)

print("Fehler Mittelwert nach oben abgeschätzt: ", kappa_cd_mean_err)

Adiabatenkoeffizienten:  [1.17647059 1.17910448 1.86153846 1.27956989 1.80851064]
Fehler Adiabatenkoeffizienten:  [0.07276727 0.07395991 0.10503864 0.05621811 0.14214577]
Mittelwert Adiabatenkoeffizient:  1.461038811631337
Fehler Mittelwert:  0.137772805878214
Fehler Mittelwert nach oben abgeschätzt:  0.1645782956880795


Unser Ergebnis ist also: <br>
$\kappa_{cd,Luft} = 1.46 \pm 0.16$

## 2. Bestimmung des Adiabatenkoeffizienten von Luft und Argon nach Rüchardt

In [4]:
t_Luft = 48.90 # Sekunden
t_Argon = 46.97 # Sekunden
t_err = 0.6 # Sekunden, Reaktionszeit

m_Luft = 0.026116 # kg
m_Luft_err = 0.000002 # kg
m_Argon = 0.026006 # kg
m_Argon_err = 0.000002 # kg

V_Luft = 5370 / (100*100*100) # m^3
V_Luft_err = 5 / (100*100*100) # m^3
V_Argon = 5460 / (100*100*100) # m^3
V_Argon_err = 5 / (100*100*100) # m^3

r_Luft = 15.95 / 1000 / 2 # m
r_Luft_err = 0.02 / 1000 / 2 # m
r_Argon = 15.97 / 1000 / 2 # m
r_Argon_err = 0.05 / 1000 / 2 # m

p_0 = 101080 # Pa
p_0_err = 10 # Pa

p_Luft = p_0 + m_Luft * 9.81 / (np.pi * r_Luft**2)
p_Luft_err = np.sqrt( ( p_0_err )**2 + ( m_Luft_err * 9.81 / (np.pi * r_Luft**2) )**2 + ( 2 * m_Luft * 9.81 * r_Luft_err / (np.pi * r_Luft**3) )**2 )
p_Argon =p_0 + m_Argon * 9.81 / (np.pi * r_Argon**2)
p_Argon_err =np.sqrt( ( p_0_err )**2 + ( m_Argon_err * 9.81 / (np.pi * r_Argon**2) )**2 + ( 2 * m_Argon * 9.81 * r_Argon_err / (np.pi * r_Argon**3) )**2 )

print("Druck Luft: ", p_Luft, "+-", p_Luft_err, "Pa")
print("Druck Argon: ", p_Argon, "+-", p_Argon_err, "Pa")

Druck Luft:  102362.22550467262 +- 10.504751604073256 Pa
Druck Argon:  102353.62874409459 +- 12.791103968836067 Pa


In [5]:
# Nun bestimmen wir die Periodendauer
T_Luft = t_Luft / 50
T_Argon = t_Argon / 50
T_err = t_err / 50

print("Periodendauern: Luft:", T_Luft, "[s]", "Argon:", T_Argon, "[s]")
print("Fehler Periodendauern: Luft:", T_err, "[s]", "Argon:", T_err, "[s]")

Periodendauern: Luft: 0.978 [s] Argon: 0.9394 [s]
Fehler Periodendauern: Luft: 0.012 [s] Argon: 0.012 [s]


In [6]:
# Wir definieren Funktionen für den Adiabatenkoeffizienten, sowie Funktionen für die Fehlerrechnung
def adiabatenkoeff(m, V, r, T, p):
    return (4*m*V)/(r**4 * T**2 * p)

def errorFrac(x, x_err, p):
    return (x_err * p)/x

def adiabatenkoefferr(m_errfrac, V_errfrac, r_errfrac, T_errfrac, p_errfrac, adiabatenkoeff):
    return np.sqrt(m_errfrac**2 + V_errfrac**2 + r_errfrac**2 + T_errfrac**2 + p_errfrac**2) * adiabatenkoeff

# Wir berechnen die Werte
kappa_r_Luft = adiabatenkoeff(m_Luft, V_Luft, r_Luft, T_Luft, p_Luft)
kappa_r_Argon = adiabatenkoeff(m_Argon, V_Argon, r_Argon, T_Argon, p_Argon)

print("Adiabatenkoeffizient Luft: ", kappa_r_Luft)
print("Adiabatenkoeffizient Argon: ", kappa_r_Argon)

Adiabatenkoeffizient Luft:  1.416448343846403
Adiabatenkoeffizient Argon:  1.5467572286371283


In [7]:
# Nun die Fehlerrechnung
m_errfrac_Luft = errorFrac(m_Luft, m_Luft_err, 1)
m_errfrac_Argon = errorFrac(m_Argon, m_Argon_err, 1)

V_errfrac_Luft = errorFrac(V_Luft, V_Luft_err, 1)
V_errfrac_Argon = errorFrac(V_Argon, V_Argon_err, 1)

r_errfrac_Luft = errorFrac(r_Luft, r_Luft_err, 4)
r_errfrac_Argon = errorFrac(r_Argon, r_Argon_err, 4)

T_errfrac_Luft = errorFrac(T_Luft, T_err, 2)
T_errfrac_Argon = errorFrac(T_Argon, T_err, 2)

p_errfrac_Luft = errorFrac(p_Luft, p_Luft_err, 1)
p_errfrac_Argon = errorFrac(p_Argon, p_Argon_err, 1)

kappa_r_Luft_err = adiabatenkoefferr(m_errfrac_Luft,V_errfrac_Luft,r_errfrac_Luft,T_errfrac_Luft,p_errfrac_Luft,kappa_r_Luft)
kappa_r_Argon_err = adiabatenkoefferr(m_errfrac_Argon,V_errfrac_Argon,r_errfrac_Argon,T_errfrac_Argon,p_errfrac_Argon,kappa_r_Argon)

print("Fehler Adiab. Luft: ", kappa_r_Luft_err)
print("Fehler Adiab. Argon: ", kappa_r_Argon_err)

Fehler Adiab. Luft:  0.03550304263655772
Fehler Adiab. Argon:  0.04403260551764061


Damit sind unsere Ergebnisse: <br>
$\kappa_{r,Luft} = 1.42 \pm 0.04 $ <br>
$\kappa_{r,Argon} = 1.55 \pm 0.04 $

## 3. Vergleichen der Werte
Wir bestimmen die $\sigma$-Abweichungen der Werte untereinander und mit den Literaturwerten.

In [8]:
kappa_Luft_lit = 1.4
kappa_Argon_lit = 1.648

sigma_cd_Luft_r_Luft = (kappa_cd_mean - kappa_r_Luft)/(np.sqrt(kappa_cd_mean_err**2 + kappa_r_Luft_err**2))

print("Abweichung der von uns berechneten Werte aus beiden Methoden: ", sigma_cd_Luft_r_Luft)

sigma_cd_Luft_lit = (kappa_cd_mean - kappa_Luft_lit)/kappa_cd_mean_err

print("Abweichung Clémont-Desormes Wert (Luft) und Literaturwert: ", sigma_cd_Luft_lit)

sigma_r_Luft_lit = (kappa_r_Luft - kappa_Luft_lit)/kappa_r_Luft_err

print("Abweichung Rüchardt Wert (Luft) und Literaturwert: ", sigma_r_Luft_lit)

sigma_r_Argon_lit = (kappa_Argon_lit - kappa_r_Argon)/kappa_r_Argon_err

print("Abweichung Rüchardt Wert (Argon) und Literaturwert: ", sigma_r_Argon_lit)

Abweichung der von uns berechneten Werte aus beiden Methoden:  0.2648454089293676
Abweichung Clémont-Desormes Wert (Luft) und Literaturwert:  0.3708800809738744
Abweichung Rüchardt Wert (Luft) und Literaturwert:  0.46329392144734594
Abweichung Rüchardt Wert (Argon) und Literaturwert:  2.2992682393575627


Unsere Messwerte stimmen also alle innerhalb der Fehlergrenzen überein.